## Representaciones vectoriales (Embeddings)

En nuestro ejemplo anterior, trabajamos con vectores de bolsa de palabras (bag-of-words) de alta dimensión con una longitud de `vocab_size`, y convertimos explícitamente vectores de representación posicional de baja dimensión en representaciones dispersas de tipo one-hot. Esta representación one-hot no es eficiente en términos de memoria. Además, cada palabra se trata de manera independiente, por lo que los vectores codificados en one-hot no expresan similitudes semánticas entre palabras.

En esta unidad, continuaremos explorando el conjunto de datos **News AG**. Para comenzar, carguemos los datos y obtengamos algunas definiciones de la unidad anterior.


In [2]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

### ¿Qué es un embedding?

La idea de un **embedding** es representar palabras utilizando vectores densos de menor dimensión que reflejen el significado semántico de la palabra. Más adelante discutiremos cómo construir embeddings de palabras significativos, pero por ahora pensemos en los embeddings como una forma de reducir la dimensionalidad de un vector de palabras.

Entonces, una capa de embedding toma una palabra como entrada y produce un vector de salida con un `embedding_size` especificado. En cierto sentido, es muy similar a una capa `Dense`, pero en lugar de tomar un vector codificado en one-hot como entrada, puede tomar un número que representa la palabra.

Al usar una capa de embedding como la primera capa en nuestra red, podemos cambiar de un modelo de bolsa de palabras a un modelo de **embedding bag**, donde primero convertimos cada palabra de nuestro texto en el embedding correspondiente y luego calculamos alguna función de agregación sobre todos esos embeddings, como `sum`, `average` o `max`.

![Imagen que muestra un clasificador con embeddings para cinco palabras de una secuencia.](../../../../../lessons/5-NLP/14-Embeddings/images/embedding-classifier-example.png)

Nuestra red neuronal clasificador consta de las siguientes capas:

* Capa `TextVectorization`, que toma una cadena como entrada y produce un tensor de números de tokens. Especificaremos un tamaño de vocabulario razonable `vocab_size` e ignoraremos las palabras que se usan con menos frecuencia. La forma de entrada será 1, y la forma de salida será $n$, ya que obtendremos $n$ tokens como resultado, cada uno de ellos conteniendo números entre 0 y `vocab_size`.
* Capa `Embedding`, que toma $n$ números y reduce cada número a un vector denso de una longitud dada (100 en nuestro ejemplo). Así, el tensor de entrada con forma $n$ se transformará en un tensor de $n\times 100$.
* Capa de agregación, que toma el promedio de este tensor a lo largo del primer eje, es decir, calculará el promedio de todos los $n$ tensores de entrada correspondientes a diferentes palabras. Para implementar esta capa, usaremos una capa `Lambda` y le pasaremos la función para calcular el promedio. La salida tendrá una forma de 100 y será la representación numérica de toda la secuencia de entrada.
* Clasificador lineal final con una capa `Dense`.


In [3]:
vocab_size = 30000
batch_size = 128

vectorizer = keras.layers.experimental.preprocessing.TextVectorization(max_tokens=vocab_size,input_shape=(1,))

model = keras.models.Sequential([
    vectorizer,    
    keras.layers.Embedding(vocab_size,100),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 100)         3000000   
                                                                 
 lambda (Lambda)             (None, 100)               0         
                                                                 
 dense (Dense)               (None, 4)                 404       
                                                                 
Total params: 3,000,404
Trainable params: 3,000,404
Non-trainable params: 0
_________________________________________________________________


En el resumen impreso, en la columna **forma de salida**, la primera dimensión del tensor `None` corresponde al tamaño del minibatch, y la segunda corresponde a la longitud de la secuencia de tokens. Todas las secuencias de tokens en el minibatch tienen longitudes diferentes. Hablaremos sobre cómo manejar esto en la próxima sección.

Ahora entrenemos la red:


In [4]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

print("Training vectorizer")
vectorizer.adapt(ds_train.take(500).map(extract_text))

model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),validation_data=ds_test.map(tupelize).batch(batch_size))

Training vectorizer
938/938 [==============================] - 20s 20ms/step - loss: 0.7891 - acc: 0.8155 - val_loss: 0.4470 - val_acc: 0.8642


> **Nota** que estamos construyendo el vectorizador basado en un subconjunto de los datos. Esto se hace para acelerar el proceso, y podría resultar en una situación en la que no todos los tokens de nuestro texto estén presentes en el vocabulario. En este caso, esos tokens serían ignorados, lo que podría resultar en una precisión ligeramente menor. Sin embargo, en la vida real, un subconjunto de texto a menudo proporciona una buena estimación del vocabulario.


### Cómo manejar tamaños variables de secuencias

Vamos a entender cómo ocurre el entrenamiento en minibatches. En el ejemplo anterior, el tensor de entrada tiene dimensión 1, y usamos minibatches de tamaño 128, por lo que el tamaño real del tensor es $128 \times 1$. Sin embargo, el número de tokens en cada oración es diferente. Si aplicamos la capa `TextVectorization` a una sola entrada, el número de tokens devueltos será diferente, dependiendo de cómo se tokenice el texto:


In [5]:
print(vectorizer('Hello, world!'))
print(vectorizer('I am glad to meet you!'))

tf.Tensor([ 1 45], shape=(2,), dtype=int64)
tf.Tensor([ 112 1271    1    3 1747  158], shape=(6,), dtype=int64)


Sin embargo, cuando aplicamos el vectorizador a varias secuencias, tiene que producir un tensor de forma rectangular, por lo que llena los elementos no utilizados con el token PAD (que en nuestro caso es cero):


In [6]:
vectorizer(['Hello, world!','I am glad to meet you!'])

<tf.Tensor: shape=(2, 6), dtype=int64, numpy=
array([[   1,   45,    0,    0,    0,    0],
       [ 112, 1271,    1,    3, 1747,  158]], dtype=int64)>

Aquí podemos ver las incrustaciones:


In [7]:
model.layers[1](vectorizer(['Hello, world!','I am glad to meet you!'])).numpy()

array([[[ 1.53059261e-02,  6.80514947e-02,  3.14026810e-02, ...,
         -8.92002955e-02,  1.52911525e-04, -5.65562584e-02],
        [ 2.57456154e-01,  2.79364467e-01, -2.03605562e-01, ...,
         -2.07474351e-01,  8.31158683e-02, -2.03911960e-01],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02],
        [ 3.98201384e-02, -8.03454965e-03,  2.39790026e-02, ...,
         -7.18549127e-04,  2.66963355e-02, -4.30646613e-02]],

       [[ 1.89674050e-01,  2.61548996e-01, -3.67433839e-02, ...,
         -2.07366899e-01, -1.05442435e-01, -2.36952081e-01],
        [ 6.16133213e-02,  1.80511594e-01,  9.77298319e-02, ...,
         -5.46628237e-02, -1.07340455e-01, -1.06589

> **Nota**: Para minimizar la cantidad de relleno, en algunos casos tiene sentido ordenar todas las secuencias del conjunto de datos en orden de longitud creciente (o, más precisamente, por número de tokens). Esto asegurará que cada minibatch contenga secuencias de longitud similar.


## Incrustaciones semánticas: Word2Vec

En nuestro ejemplo anterior, la capa de incrustación aprendió a mapear palabras a representaciones vectoriales; sin embargo, estas representaciones no tenían un significado semántico. Sería ideal aprender una representación vectorial en la que palabras similares o sinónimos correspondan a vectores que estén cerca unos de otros en términos de alguna distancia vectorial (por ejemplo, la distancia euclidiana).

Para lograr esto, necesitamos preentrenar nuestro modelo de incrustación en una gran colección de texto utilizando una técnica como [Word2Vec](https://en.wikipedia.org/wiki/Word2vec). Está basado en dos arquitecturas principales que se utilizan para producir una representación distribuida de palabras:

 - **Bolsa de palabras continua** (CBoW), donde entrenamos el modelo para predecir una palabra a partir del contexto circundante. Dado el n-grama $(W_{-2},W_{-1},W_0,W_1,W_2)$, el objetivo del modelo es predecir $W_0$ a partir de $(W_{-2},W_{-1},W_1,W_2)$.
 - **Skip-gram continuo** es lo opuesto a CBoW. El modelo utiliza la ventana de palabras de contexto circundantes para predecir la palabra actual.

CBoW es más rápido, y aunque skip-gram es más lento, hace un mejor trabajo representando palabras poco frecuentes.

![Imagen que muestra los algoritmos CBoW y Skip-Gram para convertir palabras en vectores.](../../../../../lessons/5-NLP/14-Embeddings/images/example-algorithms-for-converting-words-to-vectors.png)

Para experimentar con la incrustación Word2Vec preentrenada en el conjunto de datos de Google News, podemos usar la biblioteca **gensim**. A continuación, encontramos las palabras más similares a 'neural'.

> **Nota:** ¡Cuando creas vectores de palabras por primera vez, descargarlos puede tomar algo de tiempo!


In [8]:
import gensim.downloader as api
w2v = api.load('word2vec-google-news-300')

In [12]:
for w,p in w2v.most_similar('neural'):
    print(f"{w} -> {p}")

neuronal -> 0.7804799675941467
neurons -> 0.7326500415802002
neural_circuits -> 0.7252851724624634
neuron -> 0.7174385190010071
cortical -> 0.6941086649894714
brain_circuitry -> 0.6923246383666992
synaptic -> 0.6699118614196777
neural_circuitry -> 0.6638563275337219
neurochemical -> 0.6555314064025879
neuronal_activity -> 0.6531826257705688


También podemos extraer la incrustación vectorial de la palabra, para usarla en el entrenamiento del modelo de clasificación. La incrustación tiene 300 componentes, pero aquí solo mostramos los primeros 20 componentes del vector para mayor claridad:


In [13]:
w2v['play'][:20]

array([ 0.01226807,  0.06225586,  0.10693359,  0.05810547,  0.23828125,
        0.03686523,  0.05151367, -0.20703125,  0.01989746,  0.10058594,
       -0.03759766, -0.1015625 , -0.15820312, -0.08105469, -0.0390625 ,
       -0.05053711,  0.16015625,  0.2578125 ,  0.10058594, -0.25976562],
      dtype=float32)

Lo grandioso de las incrustaciones semánticas es que puedes manipular la codificación vectorial basada en la semántica. Por ejemplo, podemos pedir encontrar una palabra cuya representación vectorial esté lo más cerca posible de las palabras *rey* y *mujer*, y lo más lejos posible de la palabra *hombre*:


In [14]:
w2v.most_similar(positive=['king','woman'],negative=['man'])[0]

('queen', 0.7118192911148071)

Un ejemplo anterior utiliza algo de magia interna de GenSym, pero la lógica subyacente es realmente bastante simple. Una cosa interesante sobre las incrustaciones es que puedes realizar operaciones normales de vectores en los vectores de incrustación, y eso reflejaría operaciones en los **significados** de las palabras. El ejemplo anterior puede expresarse en términos de operaciones vectoriales: calculamos el vector correspondiente a **REY-HOMBRE+MUJER** (las operaciones `+` y `-` se realizan en las representaciones vectoriales de las palabras correspondientes), y luego encontramos la palabra más cercana en el diccionario a ese vector:


In [15]:
# get the vector corresponding to kind-man+woman
qvec = w2v['king']-1.7*w2v['man']+1.7*w2v['woman']
# find the index of the closest embedding vector 
d = np.sum((w2v.vectors-qvec)**2,axis=1)
min_idx = np.argmin(d)
# find the corresponding word
w2v.index_to_key[min_idx]

'queen'

> **NOTA**: Tuvimos que añadir pequeños coeficientes a los vectores de *man* y *woman* - intenta eliminarlos para ver qué sucede.

Para encontrar el vector más cercano, utilizamos la maquinaria de TensorFlow para calcular un vector de distancias entre nuestro vector y todos los vectores en el vocabulario, y luego encontramos el índice de la palabra mínima usando `argmin`.


Aunque Word2Vec parece una excelente manera de expresar la semántica de las palabras, tiene muchas desventajas, entre las que se incluyen las siguientes:

* Tanto los modelos CBoW como skip-gram son **embeddings predictivos**, y solo toman en cuenta el contexto local. Word2Vec no aprovecha el contexto global.
* Word2Vec no considera la **morfología** de las palabras, es decir, el hecho de que el significado de una palabra puede depender de diferentes partes de la misma, como la raíz.

**FastText** intenta superar la segunda limitación y se basa en Word2Vec al aprender representaciones vectoriales para cada palabra y los n-gramas de caracteres que se encuentran dentro de cada palabra. Los valores de estas representaciones se promedian en un único vector en cada paso de entrenamiento. Aunque esto añade una gran cantidad de cálculo adicional al preentrenamiento, permite que los embeddings de palabras codifiquen información a nivel de subpalabras.

Otro método, **GloVe**, utiliza un enfoque diferente para los embeddings de palabras, basado en la factorización de la matriz de contexto de palabras. Primero, construye una gran matriz que cuenta el número de apariciones de palabras en diferentes contextos, y luego intenta representar esta matriz en dimensiones más bajas de una manera que minimice la pérdida de reconstrucción.

La biblioteca gensim admite estos embeddings de palabras, y puedes experimentar con ellos cambiando el código de carga del modelo mencionado anteriormente.


## Usar embeddings preentrenados en Keras

Podemos modificar el ejemplo anterior para prellenar la matriz en nuestra capa de embedding con embeddings semánticos, como Word2Vec. Es probable que los vocabularios del embedding preentrenado y del corpus de texto no coincidan, por lo que debemos elegir uno. Aquí exploramos las dos opciones posibles: usar el vocabulario del tokenizer y usar el vocabulario de los embeddings de Word2Vec.

### Usar el vocabulario del tokenizer

Al usar el vocabulario del tokenizer, algunas palabras del vocabulario tendrán embeddings correspondientes de Word2Vec, y otras estarán ausentes. Dado que el tamaño de nuestro vocabulario es `vocab_size`, y la longitud del vector de embedding de Word2Vec es `embed_size`, la capa de embedding estará representada por una matriz de pesos con forma `vocab_size`$\times$`embed_size`. Poblaremos esta matriz recorriendo el vocabulario:


In [9]:
embed_size = len(w2v.get_vector('hello'))
print(f'Embedding size: {embed_size}')

vocab = vectorizer.get_vocabulary()
W = np.zeros((vocab_size,embed_size))
print('Populating matrix, this will take some time...',end='')
found, not_found = 0,0
for i,w in enumerate(vocab):
    try:
        W[i] = w2v.get_vector(w)
        found+=1
    except:
        # W[i] = np.random.normal(0.0,0.3,size=(embed_size,))
        not_found+=1

print(f"Done, found {found} words, {not_found} words missing")

Embedding size: 300
Populating matrix, this will take some time...Done, found 4551 words, 784 words missing


Para las palabras que no están presentes en el vocabulario de Word2Vec, podemos dejarlas como ceros o generar un vector aleatorio.

Ahora podemos definir una capa de incrustación con pesos preentrenados:


In [10]:
emb = keras.layers.Embedding(vocab_size,embed_size,weights=[W],trainable=False)
model = keras.models.Sequential([
    vectorizer, emb,
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])

In [11]:
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(batch_size),
          validation_data=ds_test.map(tupelize).batch(batch_size))

938/938 [==============================] - 10s 10ms/step - loss: 1.1075 - acc: 0.7822 - val_loss: 0.9134 - val_acc: 0.8175


> **Nota**: Observa que configuramos `trainable=False` al crear el `Embedding`, lo que significa que no estamos reentrenando la capa de Embedding. Esto puede causar que la precisión sea ligeramente menor, pero acelera el entrenamiento.

### Usando el vocabulario de embeddings

Un problema con el enfoque anterior es que los vocabularios utilizados en TextVectorization y Embedding son diferentes. Para resolver este problema, podemos usar una de las siguientes soluciones:
* Reentrenar el modelo Word2Vec con nuestro vocabulario.
* Cargar nuestro conjunto de datos utilizando el vocabulario del modelo Word2Vec preentrenado. Los vocabularios utilizados para cargar el conjunto de datos pueden especificarse durante la carga.

El último enfoque parece más sencillo, así que vamos a implementarlo. Primero, crearemos una capa `TextVectorization` con el vocabulario especificado, tomado de los embeddings de Word2Vec:


In [12]:
vocab = list(w2v.vocab.keys())
vectorizer = keras.layers.experimental.preprocessing.TextVectorization(input_shape=(1,))
vectorizer.set_vocabulary(vocab)

La biblioteca de incrustaciones de palabras gensim contiene una función conveniente, `get_keras_embeddings`, que creará automáticamente la capa de incrustaciones correspondiente de Keras para ti.


In [13]:
model = keras.models.Sequential([
    vectorizer, 
    w2v.get_keras_embedding(train_embeddings=False),
    keras.layers.Lambda(lambda x: tf.reduce_mean(x,axis=1)),
    keras.layers.Dense(4, activation='softmax')
])
model.compile(loss='sparse_categorical_crossentropy',metrics=['acc'])
model.fit(ds_train.map(tupelize).batch(128),validation_data=ds_test.map(tupelize).batch(128),epochs=5)

Epoch 1/5
938/938 [==============================] - 20s 14ms/step - loss: 1.3377 - acc: 0.4978 - val_loss: 1.2995 - val_acc: 0.5647
Epoch 2/5
938/938 [==============================] - 10s 10ms/step - loss: 1.2587 - acc: 0.5722 - val_loss: 1.2339 - val_acc: 0.5842
Epoch 3/5
938/938 [==============================] - 10s 10ms/step - loss: 1.1980 - acc: 0.5884 - val_loss: 1.1826 - val_acc: 0.5954
Epoch 4/5
938/938 [==============================] - 12s 13ms/step - loss: 1.1503 - acc: 0.6002 - val_loss: 1.1417 - val_acc: 0.6018
Epoch 5/5
938/938 [==============================] - 11s 12ms/step - loss: 1.1120 - acc: 0.6097 - val_loss: 1.1083 - val_acc: 0.6104


Una de las razones por las que no estamos viendo una mayor precisión es porque algunas palabras de nuestro conjunto de datos faltan en el vocabulario preentrenado de GloVe y, por lo tanto, se ignoran esencialmente. Para superar esto, podemos entrenar nuestras propias incrustaciones basadas en nuestro conjunto de datos.


## Representaciones contextuales

Una limitación clave de las representaciones tradicionales de embeddings preentrenados, como Word2Vec, es el hecho de que, aunque pueden capturar parte del significado de una palabra, no pueden diferenciar entre distintos significados. Esto puede causar problemas en los modelos posteriores.

Por ejemplo, la palabra 'play' tiene diferentes significados en estas dos oraciones:
- Fui a una **obra** en el teatro.
- John quiere **jugar** con sus amigos.

Los embeddings preentrenados de los que hablamos representan ambos significados de la palabra 'play' en el mismo embedding. Para superar esta limitación, necesitamos construir embeddings basados en el **modelo de lenguaje**, que está entrenado en un gran corpus de texto y *sabe* cómo se pueden combinar las palabras en diferentes contextos. Hablar de embeddings contextuales está fuera del alcance de este tutorial, pero volveremos a ellos cuando hablemos de modelos de lenguaje en la próxima unidad.



---

**Descargo de responsabilidad**:  
Este documento ha sido traducido utilizando el servicio de traducción automática [Co-op Translator](https://github.com/Azure/co-op-translator). Aunque nos esforzamos por garantizar la precisión, tenga en cuenta que las traducciones automatizadas pueden contener errores o imprecisiones. El documento original en su idioma nativo debe considerarse como la fuente autorizada. Para información crítica, se recomienda una traducción profesional realizada por humanos. No nos hacemos responsables de malentendidos o interpretaciones erróneas que puedan surgir del uso de esta traducción.
